In [1]:
%run ../load_files.ipynb

In [2]:
%run ../data_dti_matrixes.ipynb

(140, 39)


In [3]:
%run ../y_multilevel.ipynb

,attention,memory
ID,,
01_SLD,0,0
02_MSG,0,0
03_JAC,0,0
04_NSV,0,0
05_EGR,0,0


In [4]:
y = y_values.memory

In [5]:
# Training and test subsamples

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, stratify=y)

In [6]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(112, 39)
(28, 39)
(112,)
(28,)


In [7]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

from neudataload.transformer import CombineMatrixTransformer, SpreadOutMatrixTransformer, FeatureMatrixTransformer
from neudataload.utils import all_combinations

# Pipeline configuration
pipe = Pipeline([
    ('combining', CombineMatrixTransformer(column_name='combined')),
    ('spread_out', SpreadOutMatrixTransformer(columns=['combined'])),
    ('filter', FeatureMatrixTransformer(matrix_columns=['combined'])),

    ('reduce_dim', PCA()),
    ('classify', LogisticRegression(solver='liblinear'))
])

In [11]:
# Parametrized values
N_FEATURES_OPTIONS = range(1, 50, 3)
PENALTY = ['l1', 'l2']
REGULATION_C = np.linspace(1, 100, 90)
MATRIXES = list(all_combinations(['DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX',]))
EXTRA_1D_COLUMS = [None] # + list(all_combinations(available_columns))                            


param_grid = { 
        'combining__columns': MATRIXES,
        'filter__columns': EXTRA_1D_COLUMS,
        
        'reduce_dim__n_components': N_FEATURES_OPTIONS,
        'classify__penalty': PENALTY,
        'classify__C' : REGULATION_C,
}

In [12]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

# Search the best parameter combination with Cross Validation 
cv = StratifiedKFold(n_splits=3, shuffle=True)

grid = RandomizedSearchCV(pipe, cv=cv, n_jobs=-1, param_distributions=param_grid, 
                          verbose=3,
                          scoring='accuracy',
                          n_iter = 150)


In [13]:
grid.fit(X_train, y_train)

Fitting 3 folds for each of 150 candidates, totalling 450 fits
[CV] reduce_dim__n_components=43, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD'), classify__penalty=l2, classify__C=45.49438202247191 
[CV] reduce_dim__n_components=43, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD'), classify__penalty=l2, classify__C=45.49438202247191 
[CV] reduce_dim__n_components=43, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD'), classify__penalty=l2, classify__C=45.49438202247191 
[CV] reduce_dim__n_components=4, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD'), classify__penalty=l2, classify__C=89.98876404494382 
[CV] reduce_dim__n_components=4, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD'), classify__penalty=l2, classify__C=89.98876404494382 
[CV] reduce_dim__n_components=4, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD'), classify__penalty=l2, classify__C=89.98876404494382 
[CV] reduce_dim__n_components=7, filter_

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   13.2s


[CV] reduce_dim__n_components=10, filter__columns=None, combining__columns=('DTI_L1', 'DTI_RX'), classify__penalty=l2, classify__C=19.910112359550563 
[CV]  reduce_dim__n_components=16, filter__columns=None, combining__columns=('DTI_MD', 'DTI_RX'), classify__penalty=l1, classify__C=79.97752808988764, score=0.7567567567567568, total=   2.9s
[CV] reduce_dim__n_components=10, filter__columns=None, combining__columns=('DTI_L1', 'DTI_RX'), classify__penalty=l2, classify__C=19.910112359550563 
[CV]  reduce_dim__n_components=10, filter__columns=None, combining__columns=('DTI_MD', 'DTI_RX'), classify__penalty=l1, classify__C=61.06741573033708, score=0.7368421052631579, total=   2.3s
[CV]  reduce_dim__n_components=10, filter__columns=None, combining__columns=('DTI_MD', 'DTI_RX'), classify__penalty=l1, classify__C=61.06741573033708, score=0.7567567567567568, total=   2.2s
[CV] reduce_dim__n_components=10, filter__columns=None, combining__columns=('DTI_L1', 'DTI_RX'), classify__penalty=l2, classi

[CV] reduce_dim__n_components=31, filter__columns=None, combining__columns=('DTI_MD',), classify__penalty=l1, classify__C=73.30337078651685 
[CV]  reduce_dim__n_components=49, filter__columns=None, combining__columns=('DTI_L1', 'DTI_RX'), classify__penalty=l1, classify__C=77.75280898876404, score=0.7567567567567568, total=   7.0s
[CV] reduce_dim__n_components=31, filter__columns=None, combining__columns=('DTI_MD',), classify__penalty=l1, classify__C=73.30337078651685 
[CV]  reduce_dim__n_components=49, filter__columns=None, combining__columns=('DTI_L1', 'DTI_RX'), classify__penalty=l1, classify__C=77.75280898876404, score=0.7567567567567568, total=   7.2s
[CV]  reduce_dim__n_components=49, filter__columns=None, combining__columns=('DTI_L1', 'DTI_RX'), classify__penalty=l1, classify__C=77.75280898876404, score=0.7368421052631579, total=   7.5s
[CV] reduce_dim__n_components=31, filter__columns=None, combining__columns=('DTI_MD',), classify__penalty=l1, classify__C=73.30337078651685 
[CV]

[CV] reduce_dim__n_components=7, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD'), classify__penalty=l1, classify__C=21.022471910112362 
[CV]  reduce_dim__n_components=25, filter__columns=None, combining__columns=('DTI_L1', 'DTI_RX'), classify__penalty=l1, classify__C=56.61797752808989, score=0.7567567567567568, total=   7.9s
[CV] reduce_dim__n_components=7, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD'), classify__penalty=l1, classify__C=21.022471910112362 
[CV]  reduce_dim__n_components=16, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD'), classify__penalty=l2, classify__C=24.359550561797754, score=0.8108108108108109, total=   6.5s
[CV]  reduce_dim__n_components=16, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD'), classify__penalty=l2, classify__C=24.359550561797754, score=0.7894736842105263, total=   7.1s
[CV] reduce_dim__n_components=7, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD'), classify_

...

[CV]  reduce_dim__n_components=13, filter__columns=None, combining__columns=('DTI_L1', 'DTI_RX'), classify__penalty=l1, classify__C=91.10112359550563, score=0.7567567567567568, total=   2.2s
[CV] reduce_dim__n_components=10, filter__columns=None, combining__columns=('DTI_RX',), classify__penalty=l2, classify__C=16.57303370786517 
[CV]  reduce_dim__n_components=13, filter__columns=None, combining__columns=('DTI_L1', 'DTI_RX'), classify__penalty=l1, classify__C=91.10112359550563, score=0.7567567567567568, total=   2.9s
[CV] reduce_dim__n_components=10, filter__columns=None, combining__columns=('DTI_RX',), classify__penalty=l2, classify__C=16.57303370786517 
[CV]  reduce_dim__n_components=49, filter__columns=None, combining__columns=('DTI_MD',), classify__penalty=l1, classify__C=92.21348314606742, score=0.7368421052631579, total=   6.2s
[CV] reduce_dim__n_components=10, filter__columns=None, combining__columns=('DTI_RX',), classify__penalty=l2, classify__C=16.57303370786517 
[CV]  reduce_

[CV]  reduce_dim__n_components=49, filter__columns=None, combining__columns=('DTI_L1',), classify__penalty=l2, classify__C=35.48314606741573, score=0.7567567567567568, total=   6.9s
[CV] reduce_dim__n_components=25, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX'), classify__penalty=l1, classify__C=98.88764044943821 
[CV]  reduce_dim__n_components=22, filter__columns=None, combining__columns=('DTI_L1', 'DTI_RX'), classify__penalty=l2, classify__C=76.64044943820225, score=0.7368421052631579, total=   4.3s
[CV] reduce_dim__n_components=25, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX'), classify__penalty=l1, classify__C=98.88764044943821 
[CV] reduce_dim__n_components=25, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX'), classify__penalty=l1, classify__C=98.88764044943821 
[CV]  reduce_dim__n_components=22, filter__columns=None, combining__columns=('DTI_L1', 'DTI_RX'), classify__penalty=l2, classify__C=76.640449438202

[CV]  reduce_dim__n_components=37, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD', 'DTI_RX'), classify__penalty=l2, classify__C=18.797752808988765, score=0.7837837837837838, total=   5.0s
[CV] reduce_dim__n_components=7, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX'), classify__penalty=l1, classify__C=56.61797752808989 
[CV]  reduce_dim__n_components=46, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD'), classify__penalty=l1, classify__C=53.2808988764045, score=0.7837837837837838, total=   6.6s
[CV]  reduce_dim__n_components=37, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD', 'DTI_RX'), classify__penalty=l2, classify__C=18.797752808988765, score=0.8108108108108109, total=   5.4s
[CV]  reduce_dim__n_components=37, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD', 'DTI_RX'), classify__penalty=l2, classify__C=18.797752808988765, score=0.7894736842105263, total=   6.1s
[CV]  reduce_dim__n_compo

[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed:  5.4min finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=True),
          error_score='raise',
          estimator=Pipeline(memory=None,
     steps=[('combining', CombineMatrixTransformer(column_name='combined', columns=None,
             op=<function mean at 0x7fd4081bc268>)), ('spread_out', SpreadOutMatrixTransformer(columns=['combined'], symmetric=True)), ('filter', FeatureMatrixTransformer(columns=None, matrix_columns=['combined'])), ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
          fit_params=None, iid=True, n_iter=150, n_jobs=-1,
          param_distributions={'combining__columns': [('DTI_FA',), ('DTI_L1',), ('DTI_MD',), ('DTI_RX',), ('DTI_FA', 'DTI_L1'), ('DTI_FA', 'DTI_MD'), ('DTI_FA', 'DTI_RX'), ('DTI_L1', 'DTI_MD'), ('DTI_L1', 'DTI_RX'), ('DTI_MD', 'DTI_RX'), ('DTI_FA', 'DTI_L1', 'DTI_MD'), ('DTI_FA', 'DTI_L1', 'DTI_RX'), ('DTI_FA...fy__penalty': ['l1', 'l2'], 'classify__C': array([  1.     

In [14]:
%run ../grid_results.ipynb

reduce_dim__n_components = 25
filter__columns = None
combining__columns = ('DTI_FA', 'DTI_L1', 'DTI_RX')
classify__penalty = l1
classify__C = 54.3932584269663
Test
Accuracy in test: 78.57%
             precision    recall  f1-score   support

          0       0.78      1.00      0.88        21
          1       1.00      0.14      0.25         7

avg / total       0.83      0.79      0.72        28

Train
Accuracy in train: 84.82%
             precision    recall  f1-score   support

          0       0.85      0.96      0.91        84
          1       0.82      0.50      0.62        28

avg / total       0.85      0.85      0.83       112

